In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import transformations

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_transactions = pd.read_pickle('dataframes/df_transactions.pkl')
df_fx = pd.read_pickle('dataframes/df_fx.pkl')
df_currency = pd.read_pickle('dataframes/df_currency.pkl')

In [3]:
df_transactions.head()

,CURRENCY,AMOUNT,STATE,CREATED_DATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,USER_ID,TYPE,SOURCE,ID
0,GBP,3738,COMPLETED,2015-10-11 09:05:43.016000,bar,AU,misc,7285c1ec-31d0-4022-b311-0ad9227ef7f4,CARD_PAYMENT,GAIA,5a9ee109-e9b3-4598-8dd7-587591e6a470
1,GBP,588,COMPLETED,2015-10-11 20:08:39.150000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,28d68bf4-460b-4c8e-9b95-bcda9ab596b5
2,GBP,1264,COMPLETED,2015-10-11 11:37:40.908000,NaN,UA,misc,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,CARD_PAYMENT,GAIA,1f1e8817-d40b-4c09-b718-cfc4a6f211df
3,GBP,66,REVERTED,2015-10-11 20:08:35.310000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,a7aaf78c-d201-456f-9e6d-612a795e8c32
4,GBP,968,COMPLETED,2015-10-11 02:46:47.640000,supermarket,NZ,misc,821014c5-af06-40ff-91f4-77fe7667809f,CARD_PAYMENT,GAIA,27dd99a2-5539-4ba9-876a-1a94abc2701f


In [4]:
# First convert all transactions to cash amounts:

df_currency = df_currency[df_currency['exponent'] != -1]

def convert_to_cash(amount, exponent):
    return amount / 10**exponent

CURRENCY_MAP = pd.Series(df_currency['exponent'].values, index=df_currency['currency']).to_dict()

df_transactions['AMOUNT'] = df_transactions.apply(lambda x: convert_to_cash(x['AMOUNT'], CURRENCY_MAP[x['CURRENCY']]), axis=1)
df_transactions.head()

,CURRENCY,AMOUNT,STATE,CREATED_DATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,USER_ID,TYPE,SOURCE,ID
0,GBP,37.38,COMPLETED,2015-10-11 09:05:43.016000,bar,AU,misc,7285c1ec-31d0-4022-b311-0ad9227ef7f4,CARD_PAYMENT,GAIA,5a9ee109-e9b3-4598-8dd7-587591e6a470
1,GBP,5.88,COMPLETED,2015-10-11 20:08:39.150000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,28d68bf4-460b-4c8e-9b95-bcda9ab596b5
2,GBP,12.64,COMPLETED,2015-10-11 11:37:40.908000,NaN,UA,misc,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,CARD_PAYMENT,GAIA,1f1e8817-d40b-4c09-b718-cfc4a6f211df
3,GBP,0.66,REVERTED,2015-10-11 20:08:35.310000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,a7aaf78c-d201-456f-9e6d-612a795e8c32
4,GBP,9.68,COMPLETED,2015-10-11 02:46:47.640000,supermarket,NZ,misc,821014c5-af06-40ff-91f4-77fe7667809f,CARD_PAYMENT,GAIA,27dd99a2-5539-4ba9-876a-1a94abc2701f


In [5]:
# Extract only records with CURRENCY != USD as these are the ones we wish to modify:

to_convert = df_transactions[df_transactions['CURRENCY'] != 'USD']

In [6]:
# Check that USD has indeed been filtered out. Note these are the only CCY's we need from fx_rate's CCY column.

CCYs = to_convert['CURRENCY'].unique()
CCYs

array(['GBP', 'EUR', 'PLN', 'CHF', 'DKK', 'SEK', 'NOK', 'HKD', 'NZD',
       'ILS', 'TRY', 'SGD', 'AUD', 'RON', 'ZAR', 'CAD', 'AED', 'HUF',
       'JPY', 'THB', 'QAR', 'INR', 'CZK', 'MAD', 'LTC', 'ETH', 'BTC',
       'XRP'], dtype=object)

In [7]:
# Filter down fx_rates:

df_fx = df_fx[(df_fx['BASE_CCY'] == 'USD') & (df_fx['CCY'].isin(CCYs))]

In [8]:
# Next we convert the dates to python datetime objects so we can perform date arithmetic.

# First get rid of the decimal in the seconds (such granularity is uneeded and not present in fx_rates TS):
to_convert['CREATED_DATE'] = to_convert['CREATED_DATE'].apply(lambda x: x.split('.', 1)[0])

# Convert all dates from string to datetime object.
to_convert['CREATED_DATE'] = to_convert['CREATED_DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df_fx['TS'] = df_fx['TS'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

## Simplification:

Getting the exact calculation of the nearest time (to a time granularity) is time consuming (O(m * n) where m is the size of the filtered transactions table and n is the size of the filtered fx_rates tables -- both huge tables).

After inspecting the two tables, I have verified that every single date in transactions has a corresponding exchange rate on that given day in the fx_rates table. Therefore, I will get rid of the time granularity and do a groupby on the [base_ccy, ccy, timestamp], mean-aggregating on the exchange rate. This of course is not optimal since exchange rates can vary over a single day and make a noticeable difference, but taking the mean over a single day seems like an acceptable tradeoff in order to greatly improve the time complexity of the algorithm.

In [9]:
df_fx['TS'] = df_fx['TS'].apply(lambda x: x.date())
to_convert['CREATED_DATE'] = to_convert['CREATED_DATE'].apply(lambda x: x.date())

In [10]:
# Groupby to get the rates on a day-to-day basis.

fx_grouped = df_fx.groupby(['BASE_CCY', 'CCY', 'TS'])[['RATE']].mean()

# Flatten multi-level index
fx_grouped = pd.DataFrame(fx_grouped.to_records())

fx_grouped[:10]

,BASE_CCY,CCY,TS,RATE
0,USD,AED,2015-03-03,0.268133
1,USD,AED,2015-03-04,0.272975
2,USD,AED,2015-03-05,0.272013
3,USD,AED,2015-03-06,0.272765
4,USD,AED,2015-03-07,0.274105
5,USD,AED,2015-03-08,0.270002
6,USD,AED,2015-03-09,0.268817
7,USD,AED,2015-03-10,0.275396
8,USD,AED,2015-03-11,0.272765
9,USD,AED,2015-03-12,0.273052


In [11]:
# Left join the two dataframes such that we have the rate information in to_convert

fx_grouped.rename(columns={'CCY': 'CURRENCY', 'TS' : 'CREATED_DATE'}, inplace=True)
to_convert = pd.merge(to_convert, fx_grouped, on=['CURRENCY', 'CREATED_DATE'], how='left')
to_convert.head()

,CURRENCY,AMOUNT,STATE,CREATED_DATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,USER_ID,TYPE,SOURCE,ID,BASE_CCY,RATE
0,GBP,37.38,COMPLETED,2015-10-11,bar,AU,misc,7285c1ec-31d0-4022-b311-0ad9227ef7f4,CARD_PAYMENT,GAIA,5a9ee109-e9b3-4598-8dd7-587591e6a470,USD,1.344218
1,GBP,5.88,COMPLETED,2015-10-11,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,28d68bf4-460b-4c8e-9b95-bcda9ab596b5,USD,1.344218
2,GBP,12.64,COMPLETED,2015-10-11,NaN,UA,misc,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,CARD_PAYMENT,GAIA,1f1e8817-d40b-4c09-b718-cfc4a6f211df,USD,1.344218
3,GBP,0.66,REVERTED,2015-10-11,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,a7aaf78c-d201-456f-9e6d-612a795e8c32,USD,1.344218
4,GBP,9.68,COMPLETED,2015-10-11,supermarket,NZ,misc,821014c5-af06-40ff-91f4-77fe7667809f,CARD_PAYMENT,GAIA,27dd99a2-5539-4ba9-876a-1a94abc2701f,USD,1.344218


In [12]:
# Multiply amount by rate for each transaction:

def mult(x, y):
    return x*y

to_convert['AMOUNT_USD'] = to_convert.apply(lambda x: mult(x['AMOUNT'], x['RATE']), axis=1)
to_convert.head()

,CURRENCY,AMOUNT,STATE,CREATED_DATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,USER_ID,TYPE,SOURCE,ID,BASE_CCY,RATE,AMOUNT_USD
0,GBP,37.38,COMPLETED,2015-10-11,bar,AU,misc,7285c1ec-31d0-4022-b311-0ad9227ef7f4,CARD_PAYMENT,GAIA,5a9ee109-e9b3-4598-8dd7-587591e6a470,USD,1.344218,50.246872
1,GBP,5.88,COMPLETED,2015-10-11,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,28d68bf4-460b-4c8e-9b95-bcda9ab596b5,USD,1.344218,7.904002
2,GBP,12.64,COMPLETED,2015-10-11,NaN,UA,misc,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,CARD_PAYMENT,GAIA,1f1e8817-d40b-4c09-b718-cfc4a6f211df,USD,1.344218,16.990917
3,GBP,0.66,REVERTED,2015-10-11,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,a7aaf78c-d201-456f-9e6d-612a795e8c32,USD,1.344218,0.887184
4,GBP,9.68,COMPLETED,2015-10-11,supermarket,NZ,misc,821014c5-af06-40ff-91f4-77fe7667809f,CARD_PAYMENT,GAIA,27dd99a2-5539-4ba9-876a-1a94abc2701f,USD,1.344218,13.012031


In [13]:
# Finally merge this information into original dataframe:

df_transactions = pd.merge(df_transactions, to_convert, on=['ID'], how='left')
df_transactions['AMOUNT_USD'].fillna(df_transactions['AMOUNT_x'], inplace=True)

In [14]:
df_transactions = df_transactions[['STATE_x', 'CREATED_DATE_x', 'USER_ID_x', 'AMOUNT_USD']]
df_transactions.rename(columns={'STATE_x': 'STATE', 'CREATED_DATE_x': 'CREATED_DATE', 'USER_ID_x': 'USER_ID'}, inplace=True)
df_transactions.head()

,STATE,CREATED_DATE,USER_ID,AMOUNT_USD
0,COMPLETED,2015-10-11 09:05:43.016000,7285c1ec-31d0-4022-b311-0ad9227ef7f4,50.246872
1,COMPLETED,2015-10-11 20:08:39.150000,20100a1d-12bc-41ed-a5e1-bc46216e9696,7.904002
2,COMPLETED,2015-10-11 11:37:40.908000,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,16.990917
3,REVERTED,2015-10-11 20:08:35.310000,20100a1d-12bc-41ed-a5e1-bc46216e9696,0.887184
4,COMPLETED,2015-10-11 02:46:47.640000,821014c5-af06-40ff-91f4-77fe7667809f,13.012031


In [15]:
# Now we finally do the desired query to find users whose first purchase was succesful and greater than $10:

In [16]:
# Here we do a groupby on USER_ID and then sort be date of transactions and take the first one.

first_transactions = df_transactions.sort_values('CREATED_DATE').groupby('USER_ID', as_index=False).first()
first_transactions.head(20)

,USER_ID,STATE,CREATED_DATE,AMOUNT_USD
0,000e88bb-d302-4fdc-b757-2b1a2c33e7d6,REVERTED,2018-03-07 19:34:44.503000,0.161032
1,001032e0-8071-4baf-95b9-e50214665c2e,REVERTED,2017-03-08 14:49:44.527000,1.174712
2,00131af8-66f0-4526-8b5f-dc2fdb26c7d7,REVERTED,2018-06-06 21:06:16.590000,1.177144
3,001926be-3245-43fa-86dd-b40ee160b6f9,REVERTED,2017-02-26 14:05:09.970000,1.343137
4,001cc034-5730-47c6-a70c-25f42249c9ee,REVERTED,2018-07-19 15:56:17.050000,0.249730
5,0022f893-47c7-4da0-96df-7ea564bfd50d,REVERTED,2017-12-30 01:31:00.521000,1.343303
6,002ad534-53c5-4320-a199-45a2b0a9265a,REVERTED,2017-07-19 19:25:44.608000,1.344902
7,0031da48-f009-4fde-8288-e8ec96726b0b,REVERTED,2017-10-10 06:30:10.265000,1.340986
8,0045db3e-c223-4199-b069-23c9b01c336e,REVERTED,2017-11-05 19:55:06.873000,1.175266
9,0046245a-e756-42f9-a500-6512dfc1867b,REVERTED,2018-02-28 21:37:55.746000,0.249843


In [17]:
# Next we simply filter to see which ones went through (i.e. STATE == COMPLETED) and which have value over $10.
# This is the end result.

fin = first_transactions[(first_transactions['STATE'] == 'COMPLETED') & (first_transactions['AMOUNT_USD'] >= 10)]
fin.reset_index(inplace=True)
fin.head()

,index,USER_ID,STATE,CREATED_DATE,AMOUNT_USD
0,17,006de67f-beff-4f5f-ab87-790ce764a054,COMPLETED,2015-11-14 14:16:01.344000,13.417522
1,24,00870274-9ca1-4c1a-9db2-728e3dcc03f8,COMPLETED,2017-12-18 23:28:16.672000,13.417535
2,32,00a98617-0d8e-45d8-bca0-b67fe5d3471d,COMPLETED,2015-07-22 09:36:11.061000,26.789983
3,40,00e8e8aa-f9a6-4ff1-aa00-7db008ce9673,COMPLETED,2018-03-15 21:53:37.336000,13.442113
4,54,0180632d-7737-42af-aaf0-95c2714d7854,COMPLETED,2017-09-10 19:12:34.756000,13.349362


# Final Result

In [18]:
# Complete list of user ids of users whos first transaction was over $10 and it went through ('COMPLETED')
pd.set_option('display.max_rows', 1000)

fin['USER_ID']

0      006de67f-beff-4f5f-ab87-790ce764a054
1      00870274-9ca1-4c1a-9db2-728e3dcc03f8
2      00a98617-0d8e-45d8-bca0-b67fe5d3471d
3      00e8e8aa-f9a6-4ff1-aa00-7db008ce9673
4      0180632d-7737-42af-aaf0-95c2714d7854
5      01f36078-07dc-40fa-9b3b-cf3a724b4847
6      01fb1ad9-57d4-4f84-aeae-b1c5ba330d31
7      02d3eea3-bc0a-40d1-93e0-c973be17d144
8      0336d0a6-467d-46c5-b455-3e98e6c72a88
9      0381e33c-219e-472a-989e-67911f1e0d9d
10     040b582f-3693-4368-ac11-f6c7c6e4ce5f
11     04a07e52-b1c2-4ff8-90c8-d013bbfd30a5
12     04dd7a9e-df69-4067-8f50-a8b29a5f940e
13     04f2a17c-6751-4215-be4e-14e84ff02773
14     052dbc6d-a2f2-40c6-8ece-be0889ff0232
15     05610011-fb3a-4cb0-87d6-33ce8cd0bfe1
16     05bd5215-804c-4d4d-ab4b-a183a6bf119e
17     05fce692-7793-4338-b10a-57a51d50d19d
18     069593ee-a4b1-4b1c-ad96-bd2b61c0c263
19     06bb2d68-bf61-4030-8447-9de64d3ce490
20     06e026bb-658c-4691-bbb0-7795148403e2
21     074c113e-a776-4542-9813-ebc9f7d94701
22     0755c3b9-08a8-43e7-ae14-e

In [19]:
len(fin)

995